### 1. Чтение и чистка датасета

In [1]:
%%time
import json, pickle
import pandas as pd
import numpy as np
from itertools import chain

results = pickle.load(open('chgk/results.pkl', 'rb'))
players = pd.DataFrame.from_dict(pickle.load(open('chgk/players.pkl', 'rb')), orient='index').set_index('id')
tournaments = pd.DataFrame.from_dict(pickle.load(open('chgk/tournaments.pkl', 'rb')), orient='index').set_index('id')

CPU times: user 10.7 s, sys: 1.97 s, total: 12.7 s
Wall time: 23.3 s


In [3]:
tourn_19 = tournaments[tournaments['dateStart'].str.startswith('2019-')]
tourn_20 = tournaments[tournaments['dateStart'].str.startswith('2020-')]

Соединяем датасеты турниров, игроков и результатов

In [199]:
%%time
tourn_list = []
player_list = []
team_list = []
mask_list = []
pos_list = []

for tourn_id in results:
    if (tourn_id in np.concatenate((tourn_19.index, tourn_20.index))):
        for team in results[tourn_id]:
            if 'mask' in team:
                mask = team['mask']
                position = team['position']
                team_id = team['team']['id']
                for member in team['teamMembers']:
                    tourn_list.append(tourn_id)
                    mask_list.append(mask)
                    pos_list.append(position)
                    team_list.append(team_id)
                    player_list.append(member['player']['id'])
print(len(tourn_list) == len(player_list) == len(mask_list))
df = pd.DataFrame({'tournament': tourn_list,
                   'team': team_list,
                   'position': pos_list,
                   'player': player_list,
                   'mask': mask_list
             })
df = df[~df['mask'].isna()]
df['mask_len'] = df['mask'].str.len()
df

True
CPU times: user 1.07 s, sys: 105 ms, total: 1.17 s
Wall time: 1.19 s


,tournament,team,position,player,mask,mask_len
0,4772,45556,1.0,6212,111111111011111110111111111100010010,36
1,4772,45556,1.0,18332,111111111011111110111111111100010010,36
2,4772,45556,1.0,18036,111111111011111110111111111100010010,36
3,4772,45556,1.0,22799,111111111011111110111111111100010010,36
4,4772,45556,1.0,15456,111111111011111110111111111100010010,36
...,...,...,...,...,...,...
566075,6456,69918,6.0,129706,100101101100100100010110100100010001010,39
566076,6456,69918,6.0,192901,100101101100100100010110100100010001010,39
566077,6456,63129,7.0,165962,101000100110000110001000101000010100001,39
566078,6456,63129,7.0,154624,101000100110000110001000101000010100001,39


Отфильтровываем турниры с несовпадающими масками

In [200]:
same_masks = df.groupby('tournament')['mask_len'].nunique().eq(1)
same_masks = same_masks[same_masks]
df = df[df['tournament'].isin(same_masks.index)]

Разбиваем по вопросам, чистим от неопределенных ответов

In [204]:
%%time

questions = []
for l in df['mask_len']:
    questions.extend(np.arange(1, l + 1))
    
all_q = pd.DataFrame({
    'tournament': np.repeat(df['tournament'], df['mask_len']),
    'team': np.repeat(df['team'], df['mask_len']),
    'position': np.repeat(df['position'], df['mask_len']),
    'player': np.repeat(df['player'], df['mask_len']),
    'quest_num': questions,
    'correct_ans': list(chain.from_iterable(df['mask']))
})
all_q['tourn_quest'] = all_q['tournament'].astype(str) + '_' + all_q['quest_num'].astype(str)

bad_ans = all_q[all_q['correct_ans'].isin(['?', 'X'])]
all_q = all_q[~all_q['tourn_quest'].isin(bad_ans['tourn_quest'])]
all_q.loc[:, 'correct_ans'] = all_q['correct_ans'].astype(int, copy=False)

all_q

CPU times: user 39.2 s, sys: 7.48 s, total: 46.7 s
Wall time: 47.9 s


,tournament,team,position,player,quest_num,correct_ans,tourn_quest
0,4772,45556,1.0,6212,1,1,4772_1
0,4772,45556,1.0,6212,2,1,4772_2
0,4772,45556,1.0,6212,3,1,4772_3
0,4772,45556,1.0,6212,4,1,4772_4
0,4772,45556,1.0,6212,5,1,4772_5
...,...,...,...,...,...,...,...
566079,6456,63129,7.0,224329,35,0,6456_35
566079,6456,63129,7.0,224329,36,0,6456_36
566079,6456,63129,7.0,224329,37,0,6456_37
566079,6456,63129,7.0,224329,38,0,6456_38


In [205]:
all_q.to_csv('all_q.csv', index=False)

### 2. Baseline model

Обучим стандартную логистическую регрессию на повопросном датасете

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

all_q = pd.read_csv('all_q.csv')
all_q

,tournament,team,position,player,quest_num,correct_ans,tourn_quest
0,4772,45556,1.0,6212,1,1,4772_1
1,4772,45556,1.0,6212,2,1,4772_2
2,4772,45556,1.0,6212,3,1,4772_3
3,4772,45556,1.0,6212,4,1,4772_4
4,4772,45556,1.0,6212,5,1,4772_5
...,...,...,...,...,...,...,...
20275015,6456,63129,7.0,224329,35,0,6456_35
20275016,6456,63129,7.0,224329,36,0,6456_36
20275017,6456,63129,7.0,224329,37,0,6456_37
20275018,6456,63129,7.0,224329,38,0,6456_38


In [2]:
tournaments = pd.DataFrame.from_dict(pickle.load(open('chgk/tournaments.pkl', 'rb')), orient='index').set_index('id')
tourn_19 = tournaments[tournaments['dateStart'].str.startswith('2019-')]
tourn_20 = tournaments[tournaments['dateStart'].str.startswith('2020-')]

train = all_q[all_q['tournament'].isin(tourn_19.index.values)]
test = all_q[all_q['tournament'].isin(tourn_20.index.values)]
print('Train tournaments:', train['tournament'].nunique(), 'Test tournaments:', test['tournament'].nunique())
len(train) + len(test) == len(all_q)

Train tournaments: 663 Test tournaments: 169


True

Преобразовываем к виду one-hot sparse-матрицы, обучаем обыкновенным лог-регом из склерна

In [3]:
enc = OneHotEncoder(handle_unknown='ignore')
X_train = enc.fit_transform(train[['player', 'tourn_quest']])
X_test = enc.transform(test[['player', 'tourn_quest']])

y_train = train['correct_ans']
y_test = test['correct_ans']

print(X_train.shape, X_test.shape)

(16126924, 87828) (4148096, 87828)


In [4]:
%%time
base_model = LogisticRegression()
base_model.fit(X_train, y_train)

preds = base_model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, preds)

/Users/munraito/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: user 3min 25s, sys: 6.7 s, total: 3min 32s
Wall time: 1min 21s


0.6213832015856954

In [5]:
print('train loss:', log_loss(y_train, base_model.predict_proba(X_train)[:, 1]))
print('test loss:', log_loss(y_test, base_model.predict_proba(X_test)[:, 1]))

train loss: 0.4761308477012335
test loss: 0.7032753809225558


### 3. Оценка качества бейзлайн-модели

Оценим предсказанное ранжирование с истинными положениями команд в турнирах. Если мы знаем вероятность ответа каждого игрока в текущем турнире как $p_i$, то будем считать общий вес команды как $p_t = 1 - \prod_i (1 - p_i)$ 

In [12]:
from scipy.stats import spearmanr, kendalltau
pd.options.mode.chained_assignment = None

def get_correlations(pred_test):
    # предсказанная сила
    pred_ranks = test[['tournament', 'team']]
    pred_ranks['neg_pred'] = 1 - pred_test
    pred_ranks = pred_ranks.groupby(['tournament', 'team']).prod().reset_index()
    pred_ranks['position_pred'] = pred_ranks.groupby('tournament')['neg_pred'].rank('dense')
    
    # истинные положения в турнире
    true_ranks = test[['tournament', 'team', 'position']].drop_duplicates()
    combined = pd.merge(pred_ranks, true_ranks, on=['tournament', 'team'])

    # cчитаем корреляции внутри каждого турнира, затем берем среднее
    spear_corr = []
    kend_corr = []
    for tour in combined['tournament'].unique():
        curr = combined[combined['tournament'] == tour]
        # смотрим турниры с более чем одной командой
        if len(curr) > 1:
            spear_corr.append(spearmanr(curr['position'], curr['position_pred'])[0])
            kend_corr.append(kendalltau(curr['position'], curr['position_pred'])[0])
    return np.mean(spear_corr), np.mean(kend_corr)

In [89]:
spear_corr, kend_corr = get_correlations(preds)
print('По базовой модели корреляция Кендалла:', kend_corr, 'Спирмена: ', spear_corr)

По базовой модели корреляция Кендалла: 0.5939511855120105 Спирмена:  0.7496256195017332


### 4. EM-схема

**E-шаг**: фиксируем веса игроков и вопросов, вычисляем ожидание скрытой переменной $z_{ij}$ - вероятность ответа игрока $i$ на вопрос $j$ при условии параметров модели $\theta_n$ и ответов $y$.<br>
Делаем предположение, что команда отвечает правильно, если хотя бы один из игроков знает ответ, а если отвечает неправильно - то никто из  игроков не знает ответа.

$$E(z_{ij}|y,\theta_n)=\begin{cases} 
\frac{p(z_{ij}|\theta_n)}{1-\prod_i(1-p(z_{ij}|\theta_n)}, &\text{если $y = 1$}\\
0, &\text{если $y = 0$}
\end{cases}$$
**М-шаг**: фиксируем $z_{ij}$ и минимизируем log-loss, обучая логистическую регрессию с мягкими метками. <br>
За начальное приближение $z_{ij}$ возьмем предсказания нашей бейзлайн-модели, чтобы не ждать вечность, пока ЕМ-алгоритм дойдет до них сам.

In [8]:
from scipy.special import logit, expit
from sklearn.linear_model import Ridge, LinearRegression
from copy import deepcopy

In [9]:
%%time
em_train = deepcopy(train)
em_train['player_pred'] = base_model.predict_proba(X_train)[:, 1]
# em_model = Ridge(alpha=5, solver='auto', tol=1e-1) #solver='saga', fit_intercept=False
em_model = LinearRegression()
best_spear = 0

for step in range(7):
    # E-step
    em_train['neg_pred'] = 1 - em_train['player_pred']
    em_teams = 1 - em_train.groupby(['tournament', 'team', 'quest_num'])['neg_pred'].prod()
    em_train = em_train.merge(em_teams.rename('team_pred'), left_on=['tournament', 'team', 'quest_num'], right_index=True)
    em_train['hidden_var'] = em_train['player_pred'] / em_train['team_pred']
    em_train['hidden_var'] = np.where(y_train == 0, 0, em_train['hidden_var'])
    em_train['hidden_var'] = np.clip(em_train['hidden_var'], 1e-6, 1 - 1e-6)

    # M-step 
    em_model.fit(X_train, logit(em_train['hidden_var']))
    em_train['player_pred'] = expit(em_model.predict(X_train))
    em_train = em_train.drop('team_pred',1)

    # Quality
    print('Шаг', step + 1)
    pred_test = expit(em_model.predict(X_test))
    spear_corr, kend_corr = get_correlations(pred_test)
    if spear_corr > best_spear:
        best_preds = pred_test
        best_coefs = em_model.coef_

    print('Корреляции:')
    print('Кендалл -', kend_corr, 'Спирмен -', spear_corr)

Шаг 1
Корреляции:
Кендалл - 0.6176802374040026 Спирмен - 0.7721049573199811
Шаг 2
Корреляции:
Кендалл - 0.6228469661696272 Спирмен - 0.7788997789932532
Шаг 3
Корреляции:
Кендалл - 0.624461498686591 Спирмен - 0.7802125113381424
Шаг 4
Корреляции:
Кендалл - 0.621070019539612 Спирмен - 0.7770271457057119
Шаг 5
Корреляции:
Кендалл - 0.6198134984253837 Спирмен - 0.7758902860007281
Шаг 6
Корреляции:
Кендалл - 0.619217030652486 Спирмен - 0.7747343198936933
Шаг 7
Корреляции:
Кендалл - 0.6170623578038593 Спирмен - 0.7731831139149237
CPU times: user 36min 9s, sys: 6min 27s, total: 42min 36s
Wall time: 11min 48s


По итогам запуска ЕМ-алгоритма видим, что целевые метрики растут первые 3 шага, затем качество начинает падать. Скорее всего это связано с застреванием в локальном минимуме, либо переобучением (однако добавление различных регуляризаций не помогло справиться с этой проблемой). <br>**Итоговый прирост** составил чуть больше **3%**. Зафиксируем модель с наилучшим качеством и двинемся к следующим шагам.

In [12]:
np.save('em_preds.npy', best_preds)
np.save('em_coefs.npy', best_coefs)

### 5. Рейтинг-лист турниров

Будем сортировать по средней сложности вопросов (берем "сложность" из коэффициентов обученной ЕМ-модели)

In [298]:
question_weights = dict(zip(enc.categories_[1], best_coefs[:enc.categories_[1].shape[0]]))

In [299]:
tourn_rating = train[['tournament', 'quest_num', 'tourn_quest']].drop_duplicates()
tourn_rating['quest_weight'] = tourn_rating['tourn_quest'].map(question_weights)
tourn_rating = tourn_rating.groupby('tournament')['quest_weight'].mean().reset_index()
tourn_rating = tourn_rating.merge(tournaments[['name']], left_on='tournament', right_index=True)
tourn_rating = tourn_rating.sort_values(by='quest_weight', ascending=False)

In [301]:
tourn_rating.head(25)

,tournament,quest_weight,name
121,5402,3.313383,Триптих. Осень
96,5370,3.255133,Благородный Дон Синхрон
122,5404,3.205312,Кубок МТС
61,5275,3.134509,Январское диминуэндо
120,5401,3.097540,Триптих. Лето
65,5285,3.077045,Гусарская лига. II сезон. IV этап
97,5371,3.071288,Международный Карагандинский Синхрон
67,5303,3.019603,Мемориал Дмитрия Коноваленко
95,5369,3.004184,Благородный Дон
62,5276,2.997489,Уходящая натура


In [302]:
tourn_rating.tail(25)

,tournament,quest_weight,name
469,5835,-1.621003,Кубок вМоГоТУ
616,6063,-1.712865,ВДИ - С Новым Годом!
590,5998,-1.723135,Memento memes IV
563,5964,-1.955006,Dichtenszeit-2
612,6052,-1.955790,Лёгкий Смоленск
564,5965,-1.961518,ОЧВР. 1 тур
529,5915,-2.002077,Лёгкий Смоленск
541,5937,-2.017991,Квизобрейн - глава 2. New Year edition.
611,6051,-2.043052,Трэцяя актава. Ліга нацый: Беларусь
578,5985,-2.074036,Українська ліга. Етап 1


Судя по названиям наблюдаем, что действительно более значимые турниры находятся в ТОПе, а всевозможные школьные / студенческие / перефирийные лиги внизу рейтинга.

### 6. Рейтинг-лист игроков

Снова возьмем за скилл игрока коэффициенты из наших обученных моделей, отсортируем и сравним с актуальным рейтингом с сайта.

In [64]:
import requests
def get_actual_rank(p_id):
    url = f"https://rating.chgk.info/api/players/{p_id}/rating/last"
    try:
        rank = requests.get(url).json()['rating_position']
    except:
        rank = -1
    return rank

In [37]:
players = pd.DataFrame.from_dict(pickle.load(open('chgk/players.pkl', 'rb')), orient='index').set_index('id')
em_weights = dict(zip(enc.categories_[0], best_coefs[:enc.categories_[0].shape[0]]))
base_weights = dict(zip(enc.categories_[0], base_model.coef_[0][:enc.categories_[0].shape[0]]))

In [82]:
player_rating = train[['player']].drop_duplicates().reset_index(drop=True)
# coefs from both models
player_rating['em_weight'] = player_rating['player'].map(em_weights)
player_rating['base_weight'] = player_rating['player'].map(base_weights)

#ranks
player_rating['em_rank'] = player_rating['em_weight'].rank(method='dense', ascending=False)
player_rating['base_rank'] = player_rating['base_weight'].rank(method='dense', ascending=False)

# questions and tournament counts
player_rating = player_rating.merge(train.groupby('player')['quest_num'].count().rename('quest_count'),
                                    left_on='player', right_index=True)
player_rating = player_rating.merge(train.groupby('player')['tournament'].nunique().rename('tourn_count'),
                                    left_on='player', right_index=True )
# and also names
player_rating = player_rating.merge(players, left_on='player', right_index=True)
player_rating

,player,em_weight,base_weight,em_rank,base_rank,quest_count,tourn_count,name,patronymic,surname
0,6212,5.301508,2.936658,140.0,47.0,3301,80,Юрий,Яковлевич,Выменец
1,18332,5.495881,2.986409,78.0,38.0,3780,92,Александр,Витальевич,Либер
2,18036,5.495491,3.132647,79.0,18.0,1454,33,Михаил,Ильич,Левандовский
3,22799,5.352638,3.117833,123.0,22.0,2215,50,Сергей,Игоревич,Николенко
4,15456,4.918454,2.907932,273.0,57.0,2314,51,Сергей,Владимирович,Коновалов
...,...,...,...,...,...,...,...,...,...,...
56933,217855,-0.577099,-0.322787,35381.0,25372.0,36,1,Сергей,,Пахлеванян
56934,217856,-0.577099,-0.322787,35381.0,25372.0,36,1,Арман,,Мкртчян
56935,217857,-0.577099,-0.322787,35382.0,25372.0,36,1,Давид,,Енгоян
56936,217858,-0.577099,-0.322787,35382.0,25372.0,36,1,Асмик,,Авагян


In [54]:
player_rating.sort_values(by='em_rank').head(15)[['player', 'em_rank', 'base_rank', 'tourn_count', 'quest_count']]

,player,em_rank,base_rank,tourn_count,quest_count
38375,212663,1.0,6848.0,2,72
33171,22474,2.0,4353.0,2,75
35327,201102,3.0,4273.0,2,72
40754,136300,4.0,8636.0,1,36
40755,20056,5.0,5304.0,2,72
24855,17750,6.0,8759.0,1,36
42433,211291,7.0,19823.0,1,36
5957,4270,8.0,2.0,67,2690
8222,27403,9.0,1.0,55,2176
1209,28751,10.0,3.0,95,3771


Заметим, что в топе игроков по ЕМ-модели много новичков (в базовой же модели такого перекоса не наблюдается). Сделаем жесткую отсечку в минимум 30 сыгранных турниров (руководствуясь топом сайта ЧГК) и заново отранжируем.

In [66]:
frequent_players = player_rating[player_rating['tourn_count'] > 29]
frequent_players['em_rank'] = frequent_players['em_weight'].rank(method='dense', ascending=False)
frequent_players['base_rank'] = frequent_players['base_weight'].rank(method='dense', ascending=False)
frequent_players

,player,em_weight,base_weight,em_rank,base_rank,quest_count,tourn_count,name,patronymic,surname
0,6212,5.301508,2.936658,39.0,26.0,3301,80,Юрий,Яковлевич,Выменец
1,18332,5.495881,2.986409,19.0,21.0,3780,92,Александр,Витальевич,Либер
2,18036,5.495491,3.132647,20.0,10.0,1454,33,Михаил,Ильич,Левандовский
3,22799,5.352638,3.117833,32.0,11.0,2215,50,Сергей,Игоревич,Николенко
4,15456,4.918454,2.907932,98.0,32.0,2314,51,Сергей,Владимирович,Коновалов
...,...,...,...,...,...,...,...,...,...,...
30882,184001,2.917812,1.175557,1701.0,1782.0,1557,37,Вячеслав,Игоревич,Гораш
31773,118283,2.352809,1.123894,2342.0,1879.0,1198,32,Георгий,Сергеевич,Христофоров
34405,116629,1.969447,0.469008,2728.0,2865.0,1273,33,Василий,Васильевич,Дмитриев
34460,66085,2.543316,1.286701,2134.0,1555.0,1107,30,Михаил,Юрьевич,Дианов


Сравним рейтинг наших предсказаний с непосредственным рейтингом сайта (чтобы долго не ждать, берем топ-100)

In [72]:
%%time
em_top = frequent_players.sort_values(by='em_rank').iloc[:100]
em_top['true_rank'] = em_top['player'].apply(get_actual_rank)
em_top['true_rank'] = em_top['true_rank'].astype(int)

CPU times: user 2.62 s, sys: 205 ms, total: 2.83 s
Wall time: 20.7 s


In [81]:
print('В наш топ-100 с жесткой отсечкой попало', (em_top['true_rank'] < 101).sum(), 'игроков из актуального топ-100.')
print('При этом', (em_top['true_rank'] < 501).sum(), 'игроков из нашего топ-100 входят в актуальный топ-500.')

В наш топ-100 с жесткой отсечкой попало 59 игроков из актуального топ-100.
При этом 95 игроков из нашего топ-100 входят в актуальный топ-500.


In [88]:
base_top = player_rating.sort_values(by='base_rank').iloc[:100]
base_top['true_rank'] = base_top['player'].apply(get_actual_rank)
base_top['true_rank'] = base_top['true_rank'].astype(int)

print('А в топ-100 по бейзлайн-модели без каких-либо отсечек попало', (base_top['true_rank'] < 101).sum(), 'игроков из актуального топ-100.')
print('При этом', (base_top['true_rank'] < 501).sum(), 'игроков из нашего топ-100 бейзлайн-модели входят в актуальный топ-500.')

А в топ-100 по бейзлайн-модели без каких-либо отсечек попало 52 игроков из актуального топ-100.
При этом 82 игроков из нашего топ-100 бейзлайн-модели входят в актуальный топ-500.


Попробуем теперь переобучить модель, используя априорное знание о том, что новички не попадают в топ. Только не будем полностью исключать игроков с малым кол-вом сыгранных турниров, чтобы не получить множество команд-"пустышек", а просто промаркируем их ответы как неправильные. <br>
И выберем минимум турниров для отсечки не таким жестким, например, 5.

In [98]:
y_train2 = deepcopy(y_train)
new_players = player_rating[player_rating['tourn_count'] < 6]['player']
y_train2[train['player'].isin(new_players)] = 0

И повторим весь пайплайн: инициализируемся базовой моделью, затем итеративно запускаем ЕМ-схему.

In [103]:
%%time
base_model2 = LogisticRegression()
base_model2.fit(X_train, y_train2)

CPU times: user 3min 15s, sys: 25.9 s, total: 3min 40s
Wall time: 1min 26s


/Users/munraito/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [106]:
spear_corr, kend_corr = get_correlations(base_model2.predict_proba(X_test)[:, 1])
print('По базовой модели корреляция Кендалла:', kend_corr, 'Спирмена: ', spear_corr)

По базовой модели корреляция Кендалла: 0.5885159051890257 Спирмена:  0.7422323315849286


In [105]:
%%time
em_train = deepcopy(train)
em_train['player_pred'] = base_model2.predict_proba(X_train)[:, 1]
em_model2 = LinearRegression()
best_spear = 0

for step in range(5):
    # E-step
    em_train['neg_pred'] = 1 - em_train['player_pred']
    em_teams = 1 - em_train.groupby(['tournament', 'team', 'quest_num'])['neg_pred'].prod()
    em_train = em_train.merge(em_teams.rename('team_pred'), left_on=['tournament', 'team', 'quest_num'], right_index=True)
    em_train['hidden_var'] = em_train['player_pred'] / em_train['team_pred']
    em_train['hidden_var'] = np.where(y_train == 0, 0, em_train['hidden_var'])
    em_train['hidden_var'] = np.clip(em_train['hidden_var'], 1e-6, 1 - 1e-6)

    # M-step 
    em_model2.fit(X_train, logit(em_train['hidden_var']))
    em_train['player_pred'] = expit(em_model2.predict(X_train))
    em_train = em_train.drop('team_pred',1)

    # Quality
    print('Шаг', step + 1)
    pred_test = expit(em_model2.predict(X_test))
    spear_corr, kend_corr = get_correlations(pred_test)
    if spear_corr > best_spear:
        best_preds = pred_test
        best_coefs = em_model2.coef_

    print('Корреляции:')
    print('Кендалл -', kend_corr, 'Спирмен -', spear_corr)

Шаг 1
Корреляции:
Кендалл - 0.620000867079863 Спирмен - 0.7745477296086404
Шаг 2
Корреляции:
Кендалл - 0.6249509123096575 Спирмен - 0.7807563719182637
Шаг 3
Корреляции:
Кендалл - 0.6246087046771203 Спирмен - 0.7800522366661301
Шаг 4
Корреляции:
Кендалл - 0.6211897006116904 Спирмен - 0.7772593091284501
Шаг 5
Корреляции:
Кендалл - 0.6199188709827803 Спирмен - 0.7759418168635405
CPU times: user 26min 21s, sys: 4min 44s, total: 31min 5s
Wall time: 8min 38s


In [113]:
em_weights2 = dict(zip(enc.categories_[0], best_coefs[:enc.categories_[0].shape[0]]))
player_rating['em_weight2'] = player_rating['player'].map(em_weights2)
player_rating['em_rank2'] = player_rating['em_weight2'].rank(method='dense', ascending=False)
em_top2 = player_rating.sort_values(by='em_rank2').iloc[:100]

em_top2['true_rank'] = em_top2['player'].apply(get_actual_rank)
em_top2['true_rank'] = em_top2['true_rank'].astype(int)

print('В наш топ-100 без жестких отсечек попало', (em_top2['true_rank'] < 101).sum(), 'игроков из актуального топ-100.')
print('При этом', (em_top2['true_rank'] < 501).sum(), 'игроков из нашего топ-100 входят в актуальный топ-500.')

В наш топ-100 без жестких отсечек попало 52 игроков из актуального топ-100.
При этом 63 игроков из нашего топ-100 входят в актуальный топ-500.


Получили хороший результат, сравнимый с жесткой отсечкой в топ-100, при этом не потеряв во внутритурнирных корреляциях. Можно еще сильнее увеличивать порог по сыгранным турнирам, тогда топ сильнейших игроков будет приближаться к актуальному, но так рискуем потерять в целевых метриках.

In [115]:
player_rating[player_rating['surname'] == 'Николенко']

,player,em_weight,base_weight,em_rank,base_rank,quest_count,tourn_count,name,patronymic,surname,em_weight2,em_rank2
3,22799,5.352638,3.117833,123.0,22.0,2215,50,Сергей,Игоревич,Николенко,5.575942,83.0
16696,115351,4.580451,1.115966,510.0,4373.0,72,2,Кристина,Юрьевна,Николенко,4.709147,478.0
18017,22797,3.631301,1.688725,2338.0,1634.0,298,8,Любовь,Владимировна,Николенко,3.730341,2235.0
20891,115591,1.829223,0.470523,13818.0,10192.0,399,11,Александр,Владимирович,Николенко,1.879370,13499.0
25796,190298,-2.511254,-0.443429,46811.0,27470.0,36,1,Дмитрий,Олегович,Николенко,-2.459833,46775.0
42846,179372,1.263922,0.082575,19077.0,16677.0,36,1,Олег,,Николенко,1.490201,17004.0
49922,174391,-0.243229,-0.654814,32823.0,30309.0,71,2,Антонина,Александровна,Николенко,-0.301964,33173.0


### 7. Более полная выборка турниров

Попробуем теперь учитывать и более ранние турниры. Возьмем статистику игр за последние 10 лет (максимум, который влез в память)

In [1]:
%%time
import json, pickle
import pandas as pd
import numpy as np
from itertools import chain

tournaments = pd.DataFrame.from_dict(pickle.load(open('chgk/tournaments.pkl', 'rb')), orient='index').set_index('id')
results = pickle.load(open('chgk/results.pkl', 'rb'))

CPU times: user 11.3 s, sys: 2.78 s, total: 14.1 s
Wall time: 15.3 s


Готовим датасеты

In [56]:
new_tourn = tournaments[tournaments['dateStart'] > '2010']
print('В нашу выборку вошло', round(len(new_tourn) / len(tournaments), 2) * 100, '% от всех доступных турниров')

В нашу выборку вошло 80.0 % от всех доступных турниров


In [3]:
%%time
tourn_list = []
player_list = []
team_list = []
mask_list = []
pos_list = []

for tourn_id in results:
    if tourn_id in new_tourn.index:
        for team in results[tourn_id]:
            if 'mask' in team:
                mask = team['mask']
                position = team['position']
                team_id = team['team']['id']
                for member in team['teamMembers']:
                    tourn_list.append(tourn_id)
                    mask_list.append(mask)
                    pos_list.append(position)
                    team_list.append(team_id)
                    player_list.append(member['player']['id'])
print(len(tourn_list) == len(player_list) == len(mask_list))
df = pd.DataFrame({'tournament': tourn_list,
                   'team': team_list,
                   'position': pos_list,
                   'player': player_list,
                   'mask': mask_list
             })
df = df[~df['mask'].isna()]
df['mask_len'] = df['mask'].str.len()
df

True
CPU times: user 3.84 s, sys: 292 ms, total: 4.13 s
Wall time: 4.16 s


,tournament,team,position,player,mask,mask_len
4706,583,188,1.0,3207,1111111111011101111111111001110101110011111011...,90
4707,583,188,1.0,6212,1111111111011101111111111001110101110011111011...,90
4708,583,188,1.0,29399,1111111111011101111111111001110101110011111011...,90
4709,583,188,1.0,7008,1111111111011101111111111001110101110011111011...,90
4710,583,188,1.0,20207,1111111111011101111111111001110101110011111011...,90
...,...,...,...,...,...,...
2448292,6456,69918,6.0,129706,100101101100100100010110100100010001010,39
2448293,6456,69918,6.0,192901,100101101100100100010110100100010001010,39
2448294,6456,63129,7.0,165962,101000100110000110001000101000010100001,39
2448295,6456,63129,7.0,154624,101000100110000110001000101000010100001,39


In [4]:
%%time

same_masks = df.groupby('tournament')['mask_len'].nunique().eq(1)
same_masks = same_masks[same_masks]
df = df[df['tournament'].isin(same_masks.index)]

questions = []
for l in df['mask_len']:
    questions.extend(np.arange(1, l + 1))
    
all_q = pd.DataFrame({
    'tournament': np.repeat(df['tournament'], df['mask_len']),
    'team': np.repeat(df['team'], df['mask_len']),
    'position': np.repeat(df['position'], df['mask_len']),
    'player': np.repeat(df['player'], df['mask_len']),
    'quest_num': questions,
    'correct_ans': list(chain.from_iterable(df['mask']))
})
all_q['tourn_quest'] = all_q['tournament'].astype(str) + '_' + all_q['quest_num'].astype(str)

bad_ans = all_q[all_q['correct_ans'].isin(['?', 'X'])]
all_q = all_q[~all_q['tourn_quest'].isin(bad_ans['tourn_quest'])]
all_q.loc[:, 'correct_ans'] = all_q['correct_ans'].astype(int, copy=False)

all_q

CPU times: user 2min 50s, sys: 1min 51s, total: 4min 42s
Wall time: 5min 30s


,tournament,team,position,player,quest_num,correct_ans,tourn_quest
4706,583,188,1.0,3207,1,1,583_1
4706,583,188,1.0,3207,2,1,583_2
4706,583,188,1.0,3207,3,1,583_3
4706,583,188,1.0,3207,4,1,583_4
4706,583,188,1.0,3207,5,1,583_5
...,...,...,...,...,...,...,...
2448296,6456,63129,7.0,224329,35,0,6456_35
2448296,6456,63129,7.0,224329,36,0,6456_36
2448296,6456,63129,7.0,224329,37,0,6456_37
2448296,6456,63129,7.0,224329,38,0,6456_38


In [7]:
tourn_20 = tournaments[tournaments['dateStart'] > '2020']

train = all_q[~all_q['tournament'].isin(tourn_20.index.values)]
test = all_q[all_q['tournament'].isin(tourn_20.index.values)]
print('Train tournaments:', train['tournament'].nunique(), 'Test tournaments:', test['tournament'].nunique())
len(train) + len(test) == len(all_q)

Train tournaments: 3338 Test tournaments: 169


True

Обучаем бейзлайн-модель

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

enc = OneHotEncoder(handle_unknown='ignore')
X_train = enc.fit_transform(train[['player', 'tourn_quest']])
X_test = enc.transform(test[['player', 'tourn_quest']])

y_train = train['correct_ans']
y_test = test['correct_ans']

print(X_train.shape, X_test.shape)

(79624469, 317326) (4148096, 317326)


Выбираем SAGA-оптимайзер как наиболее подходящий для больших датасетов и понижаем tolerance для ускорения сходимости 

In [11]:
%%time
base_model = LogisticRegression(solver='saga', tol=1e-1)
base_model.fit(X_train, y_train)

preds = base_model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, preds)

print('train loss:', log_loss(y_train, base_model.predict_proba(X_train)[:, 1]))
print('test loss:', log_loss(y_test, base_model.predict_proba(X_test)[:, 1]))

train loss: 0.4607793219843478
test loss: 0.784917776971025
CPU times: user 9min 43s, sys: 24.1 s, total: 10min 7s
Wall time: 10min 20s


In [13]:
spear_corr, kend_corr = get_correlations(preds)
print('По базовой модели корреляция Кендалла:', kend_corr, 'Спирмена: ', spear_corr)

По базовой модели корреляция Кендалла: 0.6099785011670265 Спирмена:  0.7680842866637234


Прирост качества относительно первой модели - около 1%

Теперь давайте попробуем учитывать изменения рейтинга со временем. Сделаем предположение, что чем новее турнир - тем он актуальнее. Для этого взвесим примеры в обучающей выборке в зависимости от года проведения

In [45]:
tournaments['year'] = pd.to_datetime(tournaments['dateStart'], utc=True).dt.year
train = train.merge(tournaments['year'], left_on='tournament', right_index=True)
# веса в зависимости от года - чем раньше, тем меньше вес
year_weights = dict(zip(np.arange(2010, 2020), np.linspace(0.1, 1, 10)))
train['weight'] = train['year'].map(year_weights)
train

,tournament,team,position,player,quest_num,correct_ans,tourn_quest,year,weight
4706,583,188,1.0,3207,1,1,583_1,2010,0.1
4706,583,188,1.0,3207,2,1,583_2,2010,0.1
4706,583,188,1.0,3207,3,1,583_3,2010,0.1
4706,583,188,1.0,3207,4,1,583_4,2010,0.1
4706,583,188,1.0,3207,5,1,583_5,2010,0.1
...,...,...,...,...,...,...,...,...,...
2447977,6349,39267,24.0,63355,32,0,6349_32,2011,0.2
2447977,6349,39267,24.0,63355,33,0,6349_33,2011,0.2
2447977,6349,39267,24.0,63355,34,0,6349_34,2011,0.2
2447977,6349,39267,24.0,63355,35,0,6349_35,2011,0.2


In [46]:
%%time
base_model2 = LogisticRegression(solver='saga', tol=1e-1)
base_model2.fit(X_train, y_train, sample_weight=train['weight'])

preds = base_model2.predict_proba(X_test)[:, 1]

print('train loss:', log_loss(y_train, base_model.predict_proba(X_train)[:, 1]))
print('test loss:', log_loss(y_test, base_model.predict_proba(X_test)[:, 1]))

train loss: 0.4607793219843478
test loss: 0.784917776971025
CPU times: user 8min 19s, sys: 23 s, total: 8min 42s
Wall time: 8min 54s


In [47]:
spear_corr, kend_corr = get_correlations(preds)
print('По модели с учетом времени корреляция Кендалла:', kend_corr, 'Спирмена: ', spear_corr)

По модели с учетом времени корреляция Кендалла: 0.6149669313835672 Спирмена:  0.7719056092358177


Получили еще небольшой прирост - 0.1-0.5%. <br> Возможно имеет смысл попробовать более сложную систему взвешивания (например, по дням, а не годам)

### Итог

Резюмируя, можно сказать, что если получится совместить все три испробованных здесь подхода (ЕМ-схема, обнуление новичков и учет даты вместе с наиболее полной выборкой турниров), и дождаться всех расчетов, то скорее всего удастся подобраться к уровню внутритурнирной корреляции в 80%, при этом сохраняя адекватный общий рейтинг игроков и турниров.